# AI 語音降噪 Colab
這個 Colab 筆記本會使用 [SpeechBrain](https://github.com/speechbrain/speechbrain) 的 MetricGAN+ 模型，使用 GPU 進行語音降噪。

載入 Google Drive 中的影音或聲音檔案，並將結果產生為聲音檔 (.wav)。

In [ ]:
#@title 1. 安裝相依套件並掛載雲端硬碟
!pip install -q speechbrain torchaudio ffmpeg-python
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 2. 設定來源檔案與輸出路徑
input_path = '/content/drive/MyDrive/input.mp4'  #@param {type:'string'}
output_path = '/content/drive/MyDrive/output.wav'  #@param {type:'string'}
use_gpu = True  #@param {type:'boolean'}


In [ ]:
#@title 3. 執行降噪
import os, subprocess, torch, torchaudio
from speechbrain.pretrained.interfaces import SpectralMaskEnhancement

def extract_audio(src, dst):
    subprocess.run(['ffmpeg','-y','-i', src, '-vn','-ac','1', dst], check=True)

tmp_wav = '/content/tmp_input.wav'
ext = os.path.splitext(input_path)[1].lower()
if ext in ['.mp4','.mov','.avi','.mkv','.flv','.webm']:
    extract_audio(input_path, tmp_wav)
    processed_input = tmp_wav
else:
    processed_input = input_path

device = 'cuda' if (use_gpu and torch.cuda.is_available()) else 'cpu'
enhancer = SpectralMaskEnhancement.from_hparams(
    source='speechbrain/metricgan-plus-voicebank',
    savedir='pretrained_models/metricgan-plus-voicebank',
    run_opts={'device': device}
)
enhanced = enhancer.enhance_file(processed_input)
torchaudio.save(output_path, enhanced.squeeze(0), 16000)
print('處理完成，結果已儲存至', output_path)